### This version adds the part for user initial  set up 

In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install flask-restful

In [ ]:
#For SSL certificate
pip install -U pyOpenSSL

In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm
#from scipy import integrate
import statistics as stats
#import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
import sys
from geopy.geocoders import Nominatim
from geopy import distance
from geopy import Point
import statistics 
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Resource allows more segreggated 
from flask import Flask, request
from flask_restful import Resource, Api
from sqlalchemy import create_engine
from json import dumps
from flask import jsonify
import json
import requests

## Dataset files 

In [3]:
### import explanation data and store it in the df_explanation file
TorontoExplanation = "..\\data\\Toronto_explanation.json"
with open(TorontoExplanation,'r') as f:
    data = f.readlines()
    data = list(map(json.loads, data))
data = data[0]
#Get all the data from the dggeata file
df_explanation = pd.DataFrame(data)

### Popularity Calculation

In [4]:
def get_three_popularity_matrix(df_original,rtrain):
    # get the list of popular items by ranking the number of reviews
    numUsers = rtrain.shape[0]
    numItems = rtrain.shape[1]
    
    dff_popular = df_original.copy()
    dff_popular = dff_popular.sort_values(by=["review_count_y"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_num_of_reviews = dff_popular["business_num_id"].tolist()
    
    # get the list of popular items by ranking average rating score
    dff_popular_rating = df_original.copy()
    dff_popular_rating = dff_popular_rating.sort_values(by=["business_stars"], ascending=False).drop_duplicates(subset = 'business_id', keep = 'first')
    popular_list_avg_stars = dff_popular_rating["business_num_id"].tolist()
    
    lst_temp = []
    for item in tqdm(range(numItems)):
        numOfUsersRated = len(rtrain.toarray()[:, item].nonzero()[0])
        if numOfUsersRated <= 50:
            lst_temp.append(item)
    popular_list_avg_stars = [x for x in popular_list_avg_stars if x not in lst_temp]
    
    # get the popularity items by using the percentage liked method(number of liked items / total items)
    predictionMatrix = np.zeros((numUsers , numItems))

    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    rtrain_array = rtrain.toarray()
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(rtrain_array[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(rtrain_array[:, item]).nonzero()[0])
#         if numOfUsersRated == 0:
        # set a threshold to filter out restaurants with very few reviews
        if numOfUsersRated <= 30:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    popular_list_liked_ratio = itemPopularity.argsort()
    
    return np.asarray(popular_list_num_of_reviews),np.asarray(popular_list_avg_stars),popular_list_liked_ratio

## All Model

In [5]:
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

#Get a UI matrix if it's not item_similarity based or else IU
def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    #inverse to IU matrix
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    #for each user or item, depends UI or IU 
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        #Get prediction/similarity score for each user 1*num or user or num of items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        #Decending accoding to similarity score, select top k
        similar_users = vector_u.argsort()[::-1][1:k+1]
        
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        
        #similar_users_weights_sum = np.sum(similar_users_weights)
        #print(similar_users_weights.shape)
        #shape: num of res * k
        similar_users_ratings = matrix_train[similar_users].toarray()
              
        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]
        #print(prediction_scores_u)
        
        
        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
        
    res = np.array(prediction_scores)
    
    if item_similarity_en:
        res = res.transpose()
    return res


#Preidction score is UI or IU?
def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    #for each user
    for user_index in tqdm(range(matrix_Train.shape[0])):
        
        #take the prediction scores for user 1 * num res
        vector_u = prediction_score[user_index]
        
        #The restuarant the user rated
        vector_train = matrix_Train[user_index]
        
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


#One hot encoding for new users, passed in 1 * number items prediction score vector
def prediction_oneHotEncode(prediction_score, initial_record):
    prediction = []

    #for each user
    #for user_index in tqdm(range(matrix_Train.shape[0])):
        
    #take the prediction scores for user 1 * num res
    vector_u =prediction_score

    #The restuarant the user rated 1 * num res
    vector_train = initial_record

    if len(vector_train.nonzero()[0]) > 0:
        vector_predict = sub_routine_modified(vector_u, vector_train)
    else:
        vector_predict = np.zeros(50, dtype=np.float32)

    prediction.append(vector_predict)

    return prediction

def sub_routine_modified(vector_u, vector_train):

    #index where non-zero
    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    #get topk + num rated res prediction score descending, top index 
    candidate_index = np.argpartition(-vector_u, -1)
    
    #sort top prediction score index in range topK+len(train_index) into vector_u`
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    
    #deleted the rated res from the topk+train_index prediction score vector for user u 
    #Delete the user rated res index from the topk+numRated index
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    #so we only include the top K prediction score here
    return vector_u

## Evaluation

In [6]:
def load_dataframe_csv(path, name):
    return pd.read_csv(path+name)

def load_numpy_csr(path, name):
    return load_npz(path+name).tocsr()

def load_numpy(path, name):
    return np.load(path+name)

def loadDict(path, fileName, trainOrTest='train'):
    # json_fileName = "{:s}.json".format(fileName)
    # Read data from file:
    if(trainOrTest == 'train'):
        dataDict = json.load(open(path+fileName))
    else:
        dataDict = json.load(open(path+fileName))
    return dataDict

## For API

In [43]:
def updateList (listTobeUpdated, listUsedToUpdate):
    tempList = listTobeUpdated.copy()
    tempList.append(listUsedToUpdate)
    listUpdate = list(set(tempList))
    
    return listUpdate

def exhaustFallBack(tempRecommendList):
    ifExhaust = False
    #print(critiqueDictionary['critiqued_Res_AccrdName_list'])
    #Exhausted recommendation list
    if(len(tempRecommendList) == 0):
        ifExhaust = True
        print('Exhausted recommendation list, no matching restaurant after current critique')
        
        #update critiqued restaurant list according to name
        #critiqueDictionary['critiqued_Res_AccrdName_list'] = updateList(critiqueDictionary['critiqued_Res_AccrdName_list'], \
        #                                                                critiqueDictionary['current_user_item_predict'][recommendIndex])
        #Update critiqued restaurant total set
        critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
    #print(critiqueDictionary['critiqued_Res_AccrdName_list'])
    return ifExhaust

def updateExtendList (listTobeUpdated, listUsedToUpdate):
    tempList = listTobeUpdated.copy()
    tempList.extend(listUsedToUpdate)
    listUpdate = list(set(tempList))
    
    return listUpdate

#This method should generate a dictionary for the recommended item, to be used for API 
def constructRestaurantDict(RecommendItem_Index, user_item_predictionMatrix,df,Id_dictionary, additionalText='', empty=False):
    restaurantInfo = {}
    if empty:
        restaurantInfo['business_id'] = "empty"
        restaurantInfo['name'] = "empty"
        restaurantInfo['cuisine'] = "empty"
        restaurantInfo['price'] = "empty"
        restaurantInfo['rating'] = "empty"
        restaurantInfo['distance'] = "empty"
        #Additional text sent back from the system 
        restaurantInfo['addText'] = "empty"
    else:
        businessId = user_item_predictionMatrix[RecommendItem_Index]
        print(businessId)
        print(Id_dictionary[businessId])
        retrieveDF = df[df['business_num_id'] == businessId]
        restaurantInfo['business_id'] = str(businessId)
        restaurantInfo['name'] = str(retrieveDF.name.unique()[0])
        restaurantInfo['cuisine'] = str(retrieveDF.categories.unique()[0])
        restaurantInfo['price'] = str(retrieveDF.price.unique()[0])
        restaurantInfo['rating'] = str(retrieveDF.business_stars.unique()[0])
        restaurantInfo['distance'] = str(Id_dictionary[businessId]) + 'km'
        #Additional text sent back from the system 
        restaurantInfo['addText'] = additionalText
        
        explanation_dict = df_explanation[df_explanation["business_id"] == str(retrieveDF.business_id.unique()[0])].explanation.values[0]
        exp_list = []
        explanation_str = ''
        for (k,v) in explanation_dict.items():
            explanation_str+=v
            exp_list.append(v)
        explanation_str = exp_list[0] + ", " + exp_list[1] + " and "+ exp_list[2]

        restaurantInfo['explanation'] = explanation_str
    
    return restaurantInfo

                
#Enter critique res list and critique res according to name if fall back to square one 
#deleted critique_Res_list=[], 
def fallBackSetup(critiqued_Res_AccrdName_list=[], critiqueDistance=2.0):
    global ID_dictionary
    global variableDictionary
    global critiqueDictionary
    global recommendResDict
    
    critique_distance = critiqueDistance
    wanted_Category_index = -1
    
    variableDictionary['critique_distance'] = critique_distance
    variableDictionary['wanted_Category_index'] = wanted_Category_index
    
    #initialize critique dictionary 
    for listName in critiqueDictionary.keys():
        critiqueDictionary[listName] = []
    
    tempCritiqueResList = [key for (key, value) in ID_dictionary.items() \
                                               if value > variableDictionary['critique_distance']]
    tempCritiqueResList.extend(critiqued_Res_AccrdName_list)
    critiqueDictionary['critique_Res_list'] = tempCritiqueResList.copy()

    #critiqueDictionary['critiqued_Res_AccrdName_list'] = critiqued_Res_AccrdName_list
    critiqueDictionary['critiqued_Res_AccrdDistance_list'] = [key for (key, value) in ID_dictionary.items() \
                                                              if value > variableDictionary['critique_distance']]
    
    critiqueDictionary['current_user_item_predict'] = [item for item in test_user_item_predict if \
                                                       item not in critiqueDictionary['critique_Res_list']]
    
    #Construct recommend business json, insert into dictionary
    recBusinessJson = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],df,ID_dictionary)
    print(recBusinessJson)
    recommendResDict['recBusinessJson'] = recBusinessJson

In [8]:
#Takes in a dictionary {business id: rating}
def constructUserItemVector(UserInitialResponse):
    #Get row lis
    rowList = [0] * len(UserInitialResponse)
    #Get col list
    colList = []
    #Get data list
    dataList = []

    for resBusinessId, rating in UserInitialResponse.items():
        print(resBusinessId, rating)
        #Append the bus_num_id as column values
        colList.append(df[df['business_id']==resBusinessId].business_num_id.unique()[0])
        dataList.append(rating)

    rows = np.array(rowList)
    cols = np.array(colList)
    data = np.array(dataList)
    print(rows, cols,data)

    #Get explicit data, dimension: 1 * # of items
    userSetUpVector = csr_matrix((data, (rows, cols)), shape=(1, IK_similarity.shape[1]))
    print(userSetUpVector)
    return userSetUpVector

In [9]:
#fallBackSetup(critiqued_Res_AccrdName_list=[1,2,5,8,10], critiqueDistance=2.0)

### Get original dataframe out of the review datastet

In [10]:
data_dir = "..\\data\\"

In [11]:
df= load_dataframe_csv(data_dir, "Dataframe.csv")

In [12]:
IC_dictionary = loadDict(data_dir, "icDictionary.json")
categoryList = list(IC_dictionary.keys())

In [45]:
I_C_matrix = load_numpy_csr(data_dir, "icmatrix.npz")

### Using entire dataset as rtrain

In [13]:
#np.allclose(rtrain_load.A, rtrain.A)
rtrain= load_numpy_csr(data_dir, "rtrain.npz")

## <font color = 'green'> Item based recommend & critique INTERNAL TESTING FOR CHECKING API</font>

In [14]:
#intersections
# yonge_and_finch = Point("43.779824, -79.415665")
# bloor_and_bathurst = Point("43.665194,-79.411208")
# queen_and_spadina = Point("43.648772,-79.396259")
# bloor_and_yonge = Point("43.670409,-79.386814")
# dundas_and_yonge = Point("43.6561,-79.3802")
# spadina_and_dundas = Point("43.653004,-79.398082")
yonge_and_finch = 'idDictionary_yongefinch'
bloor_and_bathurst = 'idDictionary_bloorbathurst.json'
queen_and_spadina = 'idDictionary_queenspadina.json'
bloor_and_yonge = 'idDictionary_blooryonge.json'
dundas_and_yonge = 'idDictionary_dundasyonge.json'
spadina_and_dundas = 'idDictionary_spadinadundas.json'

#Set intersection for test case:
intersection = bloor_and_yonge

In [23]:
IK_similarity = load_numpy(data_dir, 'IKbased_II_similarity.npy')

In [16]:
IS_dictionary = loadDict(data_dir, "isDictionary.json")
IS_dictionary = {int(old_key): val for old_key, val in IS_dictionary.items()}

In [17]:
IP_dictionary = loadDict(data_dir, "ipDictionary.json")
IP_dictionary = {int(old_key): val for old_key, val in IP_dictionary.items()}

In [18]:
ID_dictionary_load = loadDict(data_dir, intersection)
ID_dictionary_load = {int(old_key): val for old_key, val in ID_dictionary_load.items()}

### <font color='green'>1.Get user initial restaurant set up</font>

In [19]:
pickList = [4331,2970,2216,3628,1844,4972,926,3286,3356,2949,3569,3792,2766,\
     3770,2931,1789,1908,4008,1658,3372,2442,4715,2188,1813,4111,4016,\
    1058,2384, 371,4227]
business_id_list = [df[df['business_num_id']==item].business_id.unique()[0] for item in pickList]
pickBusinessDict = {}
for item in pickList:
    business_name = df[df['business_num_id']==item].name.unique()[0]
    business_id = df[df['business_num_id']==item].business_id.unique()[0]
    pickBusinessDict[business_id] = business_name

In [20]:
print('pick from the following business ids')
pickBusinessDict

pick from the following business ids


{'r_BrIgzYcwo1NAuG9dLbpg': 'Pai Northern Thai Kitchen',
 'aLcFhMe6DDJ430zelCpd2A': 'Khao San Road',
 'RtUvSWO_UZ8V3Wpj0n077w': 'KINKA IZAKAYA ORIGINAL',
 'iGEvDk6hsizigmXhDKs2Vg': 'Seven Lives Tacos Y Mariscos',
 'N93EYZy9R0sdlEvubu94ig': 'Banh Mi Boys',
 'zgQHtqX0gqMw1nlBZl2VnQ': 'Momofuku Noodle Bar',
 'B70iTJjcPkuYn8ouUewWgw': 'Gusto 101',
 'e41TP5cXZqSrz50xCBJqZw': 'Insomnia Restaurant & Lounge',
 'f5O7v_X_jCg2itqacRfxhg': 'Sansotei Ramen',
 'a8pmtlVKf7NiSLI-4KejIw': 'El Catrin Destileria',
 'h_4dPV9M9aYaBliH1Eoeeg': 'Wvrst',
 'kOFDVcnj-8fd3doIpCQ06A': "Mildred's Temple Kitchen",
 'Yl2TN9c23ZGLUBSD9ks5Uw': 'Byblos',
 'k6zmSLmYAquCpJGKNnTgSQ': 'The Stockyards',
 '_xAJZOKBMPOe47p1MphB2w': 'Fishman Lobster Clubhouse Restaurant',
 'MS-hfug4QDXqb_Mws3qlzA': 'Grand Electric',
 'O1TvPrgkK2bUo5O5aSZ7lw': 'Rol San',
 'nBl_4gw5ecGzNkHyzfii8g': 'Terroni',
 'KVpvE6pOPM9KMvak4HFsNg': 'Pizzeria Libretto',
 'fGurvC5BdOfd5MIuLUQYVA': 'Dumpling House Restaurant',
 'UxWH8zRYIBgs6Q2oykvRdw': 'Cluny B

### <font color='green'>2.Initialize, input user preference</font>

In [21]:
#Initialize user preference dictionary 'business_id': '5'
UserInitialResponse = {}
businessIdInput = ''
while businessIdInput.lower() != 'stop': 
    businessIdInput = input("Input business id that you like ").strip()
    print(businessIdInput)
    if businessIdInput== 'stop':
        break;
    elif businessIdInput not in pickBusinessDict.keys():
        print('invalid business id, RE-ENTER')
    else:
        UserInitialResponse[businessIdInput] = 5
print(UserInitialResponse)

Input business id that you like O1TvPrgkK2bUo5O5aSZ7lw
O1TvPrgkK2bUo5O5aSZ7lw
Input business id that you like f5O7v_X_jCg2itqacRfxhg
f5O7v_X_jCg2itqacRfxhg
Input business id that you like stop
stop
{'O1TvPrgkK2bUo5O5aSZ7lw': 5, 'f5O7v_X_jCg2itqacRfxhg': 5}


In [24]:
#Preprocess User Preference Input, transfers the input json into csr sparse matrix, 1 row
#Get row lis
rowList = [0] * len(UserInitialResponse)
#Get col list
colList = []
#Get data list
dataList = []

for resBusinessId, rating in UserInitialResponse.items():
    print(resBusinessId, rating)
    #Append the bus_num_id as column values
    colList.append(df[df['business_id']==resBusinessId].business_num_id.unique()[0])
    dataList.append(rating)

rows = np.array(rowList)
cols = np.array(colList)
data = np.array(dataList)
print(rows, cols,data)

#Get explicit data, dimension: 1 * # of items
userSetUpVector = csr_matrix((data, (rows, cols)), shape=(1, IK_similarity.shape[1]))
print(userSetUpVector)

O1TvPrgkK2bUo5O5aSZ7lw 5
f5O7v_X_jCg2itqacRfxhg 5
[0 0] [1908 3356] [5 5]
  (0, 1908)	5
  (0, 3356)	5


In [25]:
user_item_prediction_score = predict(userSetUpVector, 110, IK_similarity, item_similarity_en= True)
#reduce dimension 
user_item_prediction_score = user_item_prediction_score[0]

100%|████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:02<00:00, 1704.38it/s]


In [27]:
user_item_predict = prediction_oneHotEncode(user_item_prediction_score, userSetUpVector)
test_user_item_predict = list(user_item_predict[0])
test_user_item_predict[:5]

recBusinessJson = constructRestaurantDict(0, test_user_item_predict\
                                                  ,df,ID_dictionary_load)
print('Check if this is the same with the API return',recBusinessJson)
    

Check if this is the same with the API return {'business_id': '13', 'name': 'Sansotei', 'cuisine': 'Ramen', 'price': '$$', 'rating': '4.0', 'distance': '0.4km', 'addText': '', 'explanation': 'broth is amazingly tasty, service is pretty quick and excellent pork broth'}


In [41]:
ID_dictionary_load

{0: 4.7,
 1: 3.1,
 2: 20.4,
 3: 10.5,
 4: 9.1,
 5: 12.4,
 6: 3.1,
 7: 5.0,
 8: 3.8,
 9: 17.1,
 10: 25.7,
 11: 3.2,
 12: 12.2,
 13: 0.4,
 14: 3.9,
 15: 0.3,
 16: 6.1,
 17: 9.5,
 18: 3.6,
 19: 2.3,
 20: 23.9,
 21: 20.4,
 22: 3.0,
 23: 3.1,
 24: 13.7,
 25: 9.7,
 26: 11.8,
 27: 22.5,
 28: 17.0,
 29: 1.4,
 30: 1.2,
 31: 1.6,
 32: 2.4,
 33: 12.8,
 34: 3.5,
 35: 1.5,
 36: 13.5,
 37: 21.4,
 38: 14.7,
 39: 5.5,
 40: 18.2,
 41: 4.4,
 42: 2.4,
 43: 1.5,
 44: 2.9,
 45: 2.6,
 46: 2.8,
 47: 15.2,
 48: 17.3,
 49: 2.7,
 50: 3.3,
 51: 3.8,
 52: 13.0,
 53: 6.0,
 54: 6.3,
 55: 15.3,
 56: 2.4,
 57: 3.7,
 58: 8.2,
 59: 4.3,
 60: 3.2,
 61: 23.6,
 62: 5.9,
 63: 19.7,
 64: 15.9,
 65: 11.2,
 66: 21.2,
 67: 15.8,
 68: 18.9,
 69: 11.7,
 70: 4.6,
 71: 4.9,
 72: 10.4,
 73: 4.3,
 74: 6.9,
 75: 17.6,
 76: 11.7,
 77: 18.6,
 78: 16.9,
 79: 20.0,
 80: 2.7,
 81: 7.2,
 82: 3.3,
 83: 5.0,
 84: 7.5,
 85: 20.4,
 86: 11.6,
 87: 5.0,
 88: 15.4,
 89: 4.0,
 90: 2.8,
 91: 15.5,
 92: 16.6,
 93: 19.5,
 94: 20.9,
 95: 1.8,
 96: 2.3

## API initialization

In [50]:
#Directly using global variables is fine, but you can only modify dictionary variables

app = Flask(__name__)
#Add resources to be much cleaner
api = Api(app)

#Initialize variable 
#Need to initialize to have only restaurants within 2.0km recommended, restaurants distance above 2.0 will be critiqued
#critique_distance = 2.0

#list of restaurant don't wanted, initialized as restaurants distance <2.0km
#critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 
#critique_Price_list = []
#ciritiqued_Rating_list = []
#list of categories don't wanted - Accumulated 
#critique_Cat_list = []

#Used to fall back, when user inputs are conflicted, modified during the critiquing process
#critiqued_Res_AccrdName_list= []
#critiqued_Res_AccrdCuisine_list = []
#critiqued_Res_AccrdPrice_list = []
#critiqued_Res_AccrdStar_list = []
#critiqued_Res_AccrdDistance_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 

#Categories that explicitly wanted, ONLY 1 FOR NOW
#wanted_Category_index = -1

#Recommendation info
recommendIndex = 0

#To be initialized
ID_dictionary = {}

#Initial user item vector for critiquing process 
#current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]

#Construct initial recommendation
#recBusinessJson = constructRestaurantDict(recommendIndex, current_user_item_predict,df,ID_dictionary)
#print(recBusinessJson)

#Following dictionaries are for retrieval 

#Construct constant variables dictionary
variableDictionary = {
    'critique_distance' : 2.0,
    'wanted_Category_index': -1,
    'max_distance': 169
}

#A dictionary that stores all the lists related with the critiquing process
critiqueDictionary = {
    'critique_Res_list': [],
    'critique_Price_list':[],
    'critique_Cat_list': [],
    'ciritiqued_Rating_list':[],
    'critiqued_Res_AccrdStar_list': [],
    'critiqued_Res_AccrdCuisine_list':[],
    'critiqued_Res_AccrdPrice_list':[],
    'critiqued_Res_AccrdName_list': [],
    'critiqued_Res_AccrdDistance_list':[],
    'test_user_item_predict': [],
    'current_user_item_predict': []
    }

#Storing the current recommended restaurant
recommendResDict = {
        'recBusinessJson': None #recBusinessJson, to be initialized
}

intersectionDict= {'yonge_and_finch': 'idDictionary_yongefinch.json',
                   'bloor_and_bathurst': 'idDictionary_bloorbathurst.json',
                   'queen_and_spadina': 'idDictionary_queenspadina.json',
                   'bloor_and_yonge':'idDictionary_blooryonge.json',
                   'dundas_and_yonge': 'idDictionary_dundasyonge.json',
                   'spadina_and_dundas': 'idDictionary_spadinadundas.json'}

class ClearUp(Resource):
    #Request for resetting user preferences 
    def get(self):
        fallBackSetup(critiqueDictionary['critiqued_Res_AccrdName_list'], critiqueDistance=2.1)
        return {'You are Resetting the user set up': 'Not retrieving anything for you'}

class Initialize(Resource):
    def get(self):
        return {'You are calling a GET for set up:': 'Not retrieving anything for you'}
    
    #Put user initial preferences to initialize the initial user recommendation 
    #Produces test_user_item_predict, computes the recommendation results for the new user 
    #request json {business id: rating}
    def put(self):
        request_json  = request.get_json()
        print(request_json)
        
        UserInitialResponse = {}
        
        #Get a list of business ids 
        businessIds = request_json.get('init_key','none')
        
        #Construct dictionary 
        for businessId in businessIds:
            UserInitialResponse[businessId] = 5
        
        #Construct the user item rating vector 
        userSetUpVector = constructUserItemVector(UserInitialResponse)
        #Get item recommendation for user 
        user_item_prediction_score = predict(userSetUpVector, 110, IK_similarity, item_similarity_en= True)[0]
        user_item_predict = prediction_oneHotEncode(user_item_prediction_score, userSetUpVector)
        #Get the user item prediction, put into dictionary 
        test_user_item_predict = list(user_item_predict[0])
        critiqueDictionary['test_user_item_predict'] = test_user_item_predict.copy()
        
        return {'You are calling a PUT for set up, the user initialization you sent:': request_json}
    
    #Post intersection {'init_key': intersection_name}
    def post(self):
        global ID_dictionary 
        global critiqueDictionary
        
        request_json  = request.get_json()
        print(request_json)

        intersectionString = request_json.get('init_key','none')
        filePath = ""
        #find intersection name, load ID_dictionary
        for intersection in intersectionDict:
            if intersectionString == intersection:
                filePath = intersectionDict[intersection]
            
        if not filePath:
            return {'request': request_json,
                   'message': 'you need to pass in valid intersection names'}
        
        #Create ID_dictionary 
        print(filePath)
        load_dict = loadDict(data_dir, filePath)
        load_dict = {int(old_key): val for old_key, val in load_dict.items()}
        ID_dictionary = load_dict.copy()
        
        #get_ID_dictionary(df,list(set(df['business_num_id'])),intersection).copy() 
        
        
        variableDictionary['maxDistance'] = max(ID_dictionary, key=ID_dictionary.get)
        print(1)
        #create initial critique restaurant list
        critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > variableDictionary['critique_distance']] 
        print(2)
        critiqued_Res_AccrdDistance_list = critique_Res_list.copy()
        print(3)
        critiqueDictionary['critique_Res_list'] = critique_Res_list.copy()
        print(4)
        critiqueDictionary['critiqued_Res_AccrdDistance_list'] = critiqued_Res_AccrdDistance_list.copy()
        print(5)
        
        critiqueDictionary['test_user_item_predict']
        
        #Creates the rest -- waiting for put method to initialize user 
#         current_user_item_predict = [item for item in critiqueDictionary['test_user_item_predict']\
#                                      if item not in critiqueDictionary['critique_Res_list']]
        current_user_item_predict = list(set(critiqueDictionary['test_user_item_predict']) - set(critiqueDictionary['critique_Res_list']))
        
        critiqueDictionary['current_user_item_predict'] = current_user_item_predict.copy()
        print(critiqueDictionary['current_user_item_predict'])
        
        recBusinessJson = constructRestaurantDict(0, critiqueDictionary['current_user_item_predict']\
                                                  ,df,load_dict)
                                
        recommendResDict['recBusinessJson'] = recBusinessJson
        
        return {'You are calling a POST for set up, intersection point:': intersectionString}
    
class Business(Resource):

    def __init__(self):
        self.critique_Price_list = []
        self.countVar = 0
    
    def get(self):
        #return {'business name': data[0]['name']} # Fetches first column that is Employee ID
        return recommendResDict['recBusinessJson']
    
    def post(self):
        some_json  = request.get_json()
        #put json object for the critiquing process 
        #Return newly computed object 
        return {'You sent': some_json}, 201
    def put(self):
        #Get the request json
        request_json  = request.get_json()
        print('Input data:', request_json)
        
        #put json object for the critiquing process 
        critique_feature = request_json.get('feature', 'none')
        critique_posNeg = request_json.get('positiveOrNegative', 'none')
        critique_value = request_json.get('critiqueValue', 'none')
        if isinstance(critique_value, list):
            critique_value = critique_value[0]
        
        #Error Handling - Shouldn't trigger if json format passed in properly 
        if critique_feature == 'none':
            print('Critique Feature cannot be recognized')
        if critique_posNeg == 'none':
            print('Critique positive or negative category cannot be recognized')
        
        #Once user send this request, critique current item 
        #tempCritiqueRestList = critiqueDictionary['critique_Res_list'].copy()
        #tempCritiqueRestList.append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #Put back into the dictionary 
        critiqueDictionary['critique_Res_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        critiqueDictionary['critiqued_Res_AccrdName_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        
        #initialize additional text to be passed back
        additionalText = ''
        #temporary updated recommendation list
        tempRecommendList = []
        #Scenario 1 - Critique Restaurant name 
        if 'name' in critique_feature:
            
            print("Saving critiqued item: ", critiqueDictionary['current_user_item_predict'][recommendIndex])      
            #additionalText = "Saving critiqued item: " + str(critiqueDictionary['current_user_item_predict'][recommendIndex])
            additionalText = "Critiquing the current displayed item"
            #update critiqued restaurant list according to name
            #critiqueDictionary['critiqued_Res_AccrdName_list'] = updateList(critiqueDictionary['critiqued_Res_AccrdName_list'], \
            #                                                                critiqueDictionary['current_user_item_predict'][recommendIndex])
            critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
            
        #Scenario 2 - Negatively critique restaurant features 
        if ('cuisine' in critique_feature) and ('negative' in critique_posNeg):
            
            #Get current recommended restaurant cuisine type
            currentCuisineType = recommendResDict['recBusinessJson']['cuisine'].split(',')
            
            #Find the correct category name the user want to critique in current recommended item categories
            critiqueValue = [cuisine for cuisine in currentCuisineType if critique_value.strip().lower() in cuisine.lower()][0]
            
            #Retrieve critiquing category index
            critiquied_Cat_Index = IC_dictionary[critiqueValue]
            print('Saving negatively critiqued cuisine type:', critique_value, ', cuisine index: ', critiquied_Cat_Index)

            #Save the newly critiqued cuisine type
            critiqueDictionary['critique_Cat_list'] = critiqueDictionary['critique_Cat_list'] + [critiquied_Cat_Index]
            
            #Handling extreme case - critique categories previously requested 
            if critiquied_Cat_Index == variableDictionary['wanted_Category_index']:
                #Reset wanted category index
                variableDictionary['wanted_Category_index'] = -1
                print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
                additionalText = additionalText + '.You are critiquing a cuisine type you previously requested\n Fall back to initial preference'
                
                #Restting the prediction list -.-? 
                critiqueDictionary['current_user_item_predict'] = \
                                        [item for item in critiqueDictionary['test_user_item_predict'] if \
                                         item not in critique_Res_list]

            #Find list of restaurants to filter out 
            critiquedItemsList = list(I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0])
            
            #Updating the critiqued items to list
            print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
            templist = critiqueDictionary['critique_Res_list'].copy()
            templist.extend(list(critiquedItemsList))
            
            #temporary updated recommend list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                          if item not in templist]
            
            #Check if exhausted list, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, falling back to initial preference'
                print('--FALLING BACK TO INITIAL PREFERENCE--')
                fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            else:
                #Update critique restaurant list 
                critiqueDictionary['critique_Res_list'] = list(set(templist))

                #Update the list of restaurants critiqued by cuisin type
                templist = critiqueDictionary.get('critiqued_Res_AccrdCuisine_list',[]).copy()
                templist.extend(critiquedItemsList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = list(set(templist))

                #Filter out critiqued items, sequence must remain the same, update current valid set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()

            #Handle case where run out of items! - Fall Back!
            #critiqueDictionary['current_user_item_predict'], message = checkIfExhaustedList(critiqueDictionary['current_user_item_predict'], \
            #                                                                       critiqueDictionary['test_user_item_predict'], \
            #                                                                       critiqueDictionary['critique_Res_list'])
            #additionalText = additionalText + '. ' + message
        
        #Scenario 3 - Positively critique restaurant cuisine type 
        if 'cuisine' in critique_feature and 'positive' in critique_posNeg:
            #Find the correct category within all the categories list - assuming exact word typed in 
            """Add ERROR HANDLING HERE"""
            positiveCritiquiCategory = [cuisine for cuisine in categoryList if \
                                        critique_value.strip().lower() in cuisine.lower()][0]

            #Retrieve cuisine type index, 1 number  
            positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]

            #Get the preferred category index, replace existing ones
            variableDictionary['wanted_Category_index'] = positiveCritiquiedIndex

            #Retrieve items/restaurants matching item category 
            matchedResList = list(I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0])
            print('matching list:', matchedResList[:10])
            
            #additionalText = 'positive critique cuisine type' + str(positiveCritiquiCategory) + 'id:' + str(positiveCritiquiedIndex)\
            #                    +'critiquing business: ' +str(matchedResList[:5]) + '...'

            #Check if user have previously critiqued - remove it 
            if positiveCritiquiedIndex in critiqueDictionary['critique_Cat_list']:
                templist = critiqueDictionary['critique_Cat_list']
                templist.remove(positiveCritiquiedIndex)
                critiqueDictionary['critique_Cat_list'] = templist

                #Update critiqued restaurant list according to cuisine type 
                templist = critiqueDictionary['critiqued_Res_AccrdCuisine_list']
                templist.remove(matchedResList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = templist

                #UPDATE CRITIQUE RESTAURANT LIST 
                #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict']\
                                                           if item not in matchedResList or \
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]
            
            tempRecommendList = [item for item in matchedResList if item not in critiqueDictionary['critique_Res_list']]
            
            #Check if exhausted recommendation list 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:    
                #Update current valid set, make sure not in critiqued restaurant set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()
            
        #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
        #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
        if 'price' in critique_feature:
            
            #Get current price label range in 1 - 4
            currentPriceLabel = IP_dictionary[critiqueDictionary['current_user_item_predict'][recommendIndex]]
            
            #Get critique_price_list first
            templist = critiqueDictionary.get('critique_Price_list', []).copy()
            
            #Change critique value to default cheaper option 
            if critique_posNeg == 'negative':
                critique_value = 'cheaper'
            
            #Update critiqued price list
            if 'positive' in critique_posNeg and critique_value == 'more expensive':
                templist.extend([price for price in range(1,currentPriceLabel+1,1)])
                
            #Negative or positive, cheaper     
            else:     
                templist.extend([price for price in range(currentPriceLabel,5,1)])
            
            #update critiqued_price_list by puting it back to the dictionary
            critiqueDictionary['critique_Price_list'] = list(set(templist))

            #Check if critiqued all price range, if critiqued all price range 
            if len(critiqueDictionary['critique_Price_list']) == 4:
                
                #Cases where users intentionally exhaust the price list
                if currentPriceLabel==4 and critique_value == 'more expensive':
                    print('You have critiqued all price range, showing the most expensive options')
                    critiqueDictionary['critique_Price_list'] = [1,2,3]
                    #Send feedback to front end
                    additionalText = 'Showing the finest restaurant options'
                elif currentPriceLabel==1 and critique_value == 'cheaper':
                    print('You have critiqued all price range, showing the cheapest option')
                    critiqueDictionary['critique_Price_list'] = [2,3,4] 
                    additionalText = 'Showing the cheapest restaurant options'
                #Cases where users have seen all prices, and modify price preference
                elif 'positive' in critique_posNeg and critique_value == 'more expensive':
                    critiqueDictionary['critique_Price_list'] = [price for price in range(1,currentPriceLabel+1,1)]
                #Negative or positive, cheaper     
                else:     
                    critiqueDictionary['critique_Price_list']=[price for price in range(currentPriceLabel,5,1)]
                
                
                #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]

                #Clear out the restuarants critiquied by price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = []
                     
            #printing and sending message to front end, the critiquing price range
            print('Critiqing price at range:', ['$'*label for label in critiqueDictionary['critique_Price_list'] ])
            #additionalText += 'Critiqing price at range:'+str(['$'*label for label in critiqueDictionary['critique_Price_list']])
            
            #Find the list of restaurants to critique 
            listCritiqueRestaurant = [key for (key, value) in IP_dictionary.items() if \
                                       value in critiqueDictionary['critique_Price_list']]
            
            #find temporary updated critique list    
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRestaurant)
            
            tempRecommendList = [item for item in critiqueDictionary['test_user_item_predict']\
                                                               if item not in tempCritiqueList]
            
            #CHECK IF EXHAUSTED, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            
            else:
                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = tempCritiqueList.copy()

                #Record the list of restaurants critiqued so far based on price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdPrice_list'], \
                                                           listCritiqueRestaurant)
                
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')

                ##Update current valid set , filter out critiqued items, sequence must remain the same
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()
                
        #Scenario 6 negatively critique rating & positively critique rating: both goes up 
        #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
        #I will critique the ratings below the current rating or specific rating 
        if 'rating' in critique_feature:
            currentRating = float(recommendResDict['recBusinessJson']['rating'])
            
            if critique_value.lower() == 'current_rating':
                critique_value = currentRating
        
            #Critiquing the restaurants that has ratings equal and below this restaurant 
            critiqueRes = [rating/10 for rating in range(0,int(float(critique_value)*10)1,5)]
            critiqueDictionary['ciritiqued_Rating_list'] = updateExtendList(critiqueDictionary['ciritiqued_Rating_list'], \
                                                                           critiqueRes)
            #If critiqued all price
            if critiqueDictionary['ciritiqued_Rating_list'] == [price/10 for price in range(0,51,5)]:
                print('exhausted list, no better restaurants, recommending the finest restaurants')
                additionalText += '| Exhausted list, no better restaurants, recommending restaurants beyond current rating'

                #reset critiqued restaurant list 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))] 

                #Clear out the restuarants critiquied by rating 
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = []
                critiqueDictionary['ciritiqued_Rating_list'] = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

            print('critiquing restaurants at rating rangeing at:',critiqueDictionary['ciritiqued_Rating_list'])
            #additionalText += '| critiquing restaurants at rating rangeing at:'+str(critiqueDictionary['ciritiqued_Rating_list'])

            #Get the list of restaurants to critique
            listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if \
                               value in critiqueDictionary['ciritiqued_Rating_list']]
            
            #CHECK EXHAUSTED - Check to see if user critique exhausts the recommendation list
            #Take a temporary list as updated critiquing list 
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)
            
            #Take a temporary list as updated recommendation list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                               if item not in tempCritiqueList]
            #Exhausted recommendation list
            if exhaustFallBack(tempRecommendList):
                additionalText += '| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)

                #Record those as well
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdStar_list'], \
                                                                                      listCritiqueRes)
                #Update current valid set 
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        
        #Scenario #7 Critique on Distance, goes up or down, only say positive critiques. Want closer, further, or specific value 
        if 'distance' in critique_feature:
            
            #Get current distance
            currentDistance = float(recommendResDict['recBusinessJson']['distance'].strip('km'))
            
            #If want further distance restaurants
            if 'positive' in critique_posNeg and 'further' in critique_value.lower():
                print('Critiquing distance <=', currentDistance)
                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]

            #Closer distance, critique value not empty 
            else:
                #specifiying a distance 
                if 'positive' in critique_posNeg and not 'closer' in critique_value.lower():
                    currentDistance = float(critique_value)

                print('Critiquing distance >=', currentDistance)

                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]


            #Update critique restaurant list, only keep the ones that are critiqued under other features
            critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                                      (item in list(set(listCritiquedRestaurant+\
                                                                        critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))] 

            #Update list of restaurants critiqued by the restuarnt distance, entire replacement
            critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

            #Update initial valid restuarnt list
            critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in critiqueDictionary['critique_Res_list']]

            #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
            if(len(critiqueDictionary['current_user_item_predict']) == 0):
                print('Exhausted sytem')

                #new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')
                additionalText += 'No restaurants that matches your preference within this area, currently default back to 2.0km'
                new_Distance = 2.0
                
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]

                #Restore original critiqued restaurants under distance feature 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    (item in list(set(listCritiquedRestaurant + \
                                                      critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))]   
                #New critiquing restuarants
                critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

                #Update initial valid restuarnt list
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        #Full fall back 
        if(len(critiqueDictionary['current_user_item_predict']) ==0):
            fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            additionalText += ' Ran out of recommendations at the end of the process, reset, filtered out seen restaurants'
            
            #If still no results
            if len(critiqueDictionary['current_user_item_predict']) ==0:
                fallBackSetup(critique_Res_list=[], \
                              critiqued_Res_AccrdName_list=[], \
                              critiqueDistance=2.0)
                additionalText += ' Clearing EVERYTHING'

        #Construct newly returning item
        return_Json = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],\
                                              df,ID_dictionary,additionalText)

        #Update restaurant information for the GET method
        recommendResDict['recBusinessJson'] = return_Json.copy()

        #Return newly computed object 
        return {'You_sent': request_json, 'Result': return_Json}, 201
            
api.add_resource(Business, '/business') # Route_1
api.add_resource(Initialize, '/initialization') #Route_2
api.add_resource(ClearUp, '/clearup') #Route_3

if __name__ == '__main__':
     app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


{'init_key': ['nBl_4gw5ecGzNkHyzfii8g', 'q5xrVJ4kivx_yEfJeOKNYQ', 'UDk7qUVIetK3Y3NuARguqg']}
nBl_4gw5ecGzNkHyzfii8g 5
q5xrVJ4kivx_yEfJeOKNYQ 5
UDk7qUVIetK3Y3NuARguqg 5
[0 0 0] [4008 4227 2384] [5 5 5]
  (0, 2384)	5
  (0, 4008)	5
  (0, 4227)	5


100%|████████████████████████████████████████████████████████████████████████████| 4996/4996 [00:03<00:00, 1625.02it/s]
127.0.0.1 - - [20/Feb/2020 00:11:56] "PUT /initialization HTTP/1.1" 200 -


{'init_key': 'yonge_and_finch'}
idDictionary_yongefinch.json
1
2
3
4
5
[3, 3595, 12, 1549, 3088, 3607, 26, 2593, 4643, 4647, 1578, 1579, 1581, 1072, 2608, 3633, 564, 3638, 4667, 4163, 2118, 3656, 2635, 592, 4689, 4190, 2144, 3172, 1638, 2668, 3187, 4729, 3716, 1165, 2701, 1685, 1692, 157, 2716, 1696, 1187, 1702, 3240, 2738, 1715, 3763, 1206, 1723, 1726, 192, 2240, 4292, 4806, 2763, 1229, 4814, 3283, 2268, 2780, 1758, 4316, 2276, 2791, 3817, 236, 1263, 3318, 248, 250, 3325, 1792, 3844, 3335, 1293, 2839, 4375, 287, 3361, 4386, 3876, 4901, 295, 302, 2862, 1328, 3889, 2871, 1340, 1344, 1861, 4941, 2894, 1366, 2395, 865, 4450, 1379, 2916, 2917, 2925, 2926, 880, 369, 885, 3964, 4988, 4481, 2436, 4485, 1421, 1938, 920, 1432, 410, 3992, 414, 2464, 418, 934, 946, 4531, 436, 437, 4027, 4028, 1987, 964, 3524, 460, 470, 1496, 2528, 2024, 1513, 1002, 2538, 4585, 1005, 1520, 4595, 4597, 2556, 4605]
3
1.9


...

127.0.0.1 - - [20/Feb/2020 00:12:42] "PUT /business HTTP/1.1" 201 -
127.0.0.1 - - [20/Feb/2020 00:12:49] "GET /business HTTP/1.1" 200 -


Input data: {'feature': 'price', 'positiveOrNegative': 'positive', 'critiqueValue': 'cheaper'}
You have critiqued all price range, showing the cheapest option
Critiqing price at range: ['$$', '$$$', '$$$$']
Saving critiqued items at index:  [1384, 4271, 1030, 1084, 3458] ...
1421
0.5


127.0.0.1 - - [20/Feb/2020 00:13:03] "PUT /business HTTP/1.1" 201 -
127.0.0.1 - - [20/Feb/2020 00:13:12] "GET /business HTTP/1.1" 200 -


Input data: {'feature': 'rating', 'positiveOrNegative': 'positive', 'critiqueValue': 'current_rating'}
critiquing restaurants at rating rangeing at: [0.0, 0.5, 2.0, 1.0, 1.5, 2.5, 3.0, 3.5, 4.0, 4.5]
Exhausted recommendation list, no matching restaurant after current critique
1344
0.0


127.0.0.1 - - [20/Feb/2020 00:13:20] "PUT /business HTTP/1.1" 201 -


## original API version - backup

In [92]:
#Directly using global variables is fine, but you can only modify dictionary variables

app = Flask(__name__)
#Add resources to be much cleaner
api = Api(app)

#Initialize variable 

#Need to initialize to have only restaurants within 2.0km recommended, restaurants distance above 2.0 will be critiqued
critique_distance = 2.0

#list of restaurant don't wanted, initialized as restaurants distance <2.0km
critique_Res_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 
critique_Price_list = []
ciritiqued_Rating_list = []
#list of categories don't wanted - Accumulated 
critique_Cat_list = []

#Used to fall back, when user inputs are conflicted, modified during the critiquing process
critiqued_Res_AccrdName_list= []
critiqued_Res_AccrdCuisine_list = []
critiqued_Res_AccrdPrice_list = []
critiqued_Res_AccrdStar_list = []
critiqued_Res_AccrdDistance_list = [key for (key, value) in ID_dictionary.items() if value > critique_distance] 

#Categories that explicitly wanted, ONLY 1 FOR NOW
wanted_Category_index = -1

#Recommendation info
recommendIndex = 0  

#Initial user item vector for critiquing process 
current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]

#Construct initial recommendation
recBusinessJson = constructRestaurantDict(recommendIndex, current_user_item_predict,df,ID_dictionary)
print(recBusinessJson)

#Following dictionaries are for retrieval 

#Construct constant variables dictionary
variableDictionary = {
    'critique_distance' : critique_distance,
    'wanted_Category_index': wanted_Category_index
    
}

#A dictionary that stores all the lists related with the critiquing process
critiqueDictionary = {
    'critique_Res_list': critique_Res_list,
    'critique_Price_list':critique_Price_list,
    'critique_Cat_list': critique_Cat_list,
    'ciritiqued_Rating_list':ciritiqued_Rating_list,
    'critiqued_Res_AccrdStar_list': critiqued_Res_AccrdStar_list,
    'critiqued_Res_AccrdCuisine_list':critiqued_Res_AccrdCuisine_list,
    'critiqued_Res_AccrdPrice_list':critiqued_Res_AccrdPrice_list,
    'critiqued_Res_AccrdName_list': critiqued_Res_AccrdName_list,
    'critiqued_Res_AccrdDistance_list':critiqued_Res_AccrdDistance_list,
    'test_user_item_predict': test_user_item_predict,
    'current_user_item_predict': current_user_item_predict
    }

#Storing the current recommended restaurant
recommendResDict = {
        'recBusinessJson': recBusinessJson
}


class Business(Resource):

    def __init__(self):
        self.critique_Price_list = []
        self.countVar = 0
    
    def get(self):
        #return {'business name': data[0]['name']} # Fetches first column that is Employee ID
        return recommendResDict['recBusinessJson']
    
    def post(self):
        some_json  = request.get_json()
        #put json object for the critiquing process 
        #Return newly computed object 
        return {'You sent': some_json}, 201
    
    def put(self):
        #Get the request json
        request_json  = request.get_json()
        print('Input data:', request_json)
        
        #put json object for the critiquing process 
        critique_feature = request_json.get('feature', 'none')
        critique_posNeg = request_json.get('positiveOrNegative', 'none')
        critique_value = request_json.get('critiqueValue', 'none')
        if isinstance(critique_value, list):
            critique_value = critique_value[0]
        
        #Error Handling - Shouldn't trigger if json format passed in properly 
        if critique_feature == 'none':
            print('Critique Feature cannot be recognized')
        if critique_posNeg == 'none':
            print('Critique positive or negative category cannot be recognized')
        
        #Once user send this request, critique current item 
        tempCritiqueRestList = critiqueDictionary['critique_Res_list'].copy()
        tempCritiqueRestList.append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #Put back into the dictionary 
        critiqueDictionary['critique_Res_list'] = list(set(tempCritiqueRestList)).copy()
        critiqueDictionary['critiqued_Res_AccrdName_list'].append(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
        #initialize additional text to be passed back
        additionalText = ''
        #temporary updated recommendation list
        tempRecommendList = []
        
        #Scenario 1 - Critique Restaurant name 
        if 'name' in critique_feature:
            
            print("Saving critiqued item: ", critiqueDictionary['current_user_item_predict'][recommendIndex])      
            additionalText = "Saving critiqued item: " + str(critiqueDictionary['current_user_item_predict'][recommendIndex])
        
            #Update critiqued restaurant total set
            critiqueDictionary['current_user_item_predict'] = critiqueDictionary['current_user_item_predict'][recommendIndex+1 :]
            
            
        #Scenario 2 - Negatively critique restaurant features 
        if ('cuisine' in critique_feature) and ('negative' in critique_posNeg):
            
            #Get current recommended restaurant cuisine type
            currentCuisineType = recommendResDict['recBusinessJson']['cuisine'].split(',')
            
            #Find the correct category name the user want to critique in current recommended item categories
            critiqueValue = [cuisine for cuisine in currentCuisineType if critique_value.strip().lower() in cuisine.lower()][0]
            
            #Retrieve critiquing category index
            critiquied_Cat_Index = IC_dictionary[critiqueValue]
            print('Saving negatively critiqued cuisine type:', critique_value, ', cuisine index: ', critiquied_Cat_Index)

            #Save the newly critiqued cuisine type
            critiqueDictionary['critique_Cat_list'] = critiqueDictionary['critique_Cat_list'] + [critiquied_Cat_Index]
            
            #Handling extreme case - critique categories previously requested 
            if critiquied_Cat_Index == variableDictionary['wanted_Category_index']:
                #Reset wanted category index
                variableDictionary['wanted_Category_index'] = -1
                print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
                additionalText = additionalText + '.You are critiquing a cuisine type you previously requested\n Fall back to initial preference'
                
                #Restting the prediction list -.-? 
                critiqueDictionary['current_user_item_predict'] = \
                                        [item for item in critiqueDictionary['test_user_item_predict'] if \
                                         item not in critique_Res_list]

            #Find list of restaurants to filter out 
            critiquedItemsList = list(I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0])
            
            #Updating the critiqued items to list
            print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
            templist = critiqueDictionary['critique_Res_list'].copy()
            templist.extend(list(critiquedItemsList))
            
            #temporary updated recommend list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                          if item not in templist]
            
            #Check if exhausted list, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, falling back to initial preference'
                print('--FALLING BACK TO INITIAL PREFERENCE--')
                fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            else:
                #Update critique restaurant list 
                critiqueDictionary['critique_Res_list'] = list(set(templist))

                #Update the list of restaurants critiqued by cuisin type
                templist = critiqueDictionary.get('critiqued_Res_AccrdCuisine_list',[]).copy()
                templist.extend(critiquedItemsList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = list(set(templist))

                #Filter out critiqued items, sequence must remain the same, update current valid set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()

            #Handle case where run out of items! - Fall Back!
            #critiqueDictionary['current_user_item_predict'], message = checkIfExhaustedList(critiqueDictionary['current_user_item_predict'], \
            #                                                                       critiqueDictionary['test_user_item_predict'], \
            #                                                                       critiqueDictionary['critique_Res_list'])
            #additionalText = additionalText + '. ' + message
        
        #Scenario 3 - Positively critique restaurant cuisine type 
        if 'cuisine' in critique_feature and 'positive' in critique_posNeg:
            #Find the correct category within all the categories list - assuming exact word typed in 
            """Add ERROR HANDLING HERE"""
            positiveCritiquiCategory = [cuisine for cuisine in categoryList if \
                                        critique_value.strip().lower() in cuisine.lower()][0]

            #Retrieve cuisine type index, 1 number  
            positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]

            #Get the preferred category index, replace existing ones
            variableDictionary['wanted_Category_index'] = positiveCritiquiedIndex

            #Retrieve items/restaurants matching item category 
            matchedResList = list(I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0])
            print('matching list:', matchedResList[:10])
            
            #additionalText = 'positive critique cuisine type' + str(positiveCritiquiCategory) + 'id:' + str(positiveCritiquiedIndex)\
            #                    +'critiquing business: ' +str(matchedResList[:5]) + '...'

            #Check if user have previously critiqued - remove it 
            if positiveCritiquiedIndex in critiqueDictionary['critique_Cat_list']:
                templist = critiqueDictionary['critique_Cat_list']
                templist.remove(positiveCritiquiedIndex)
                critiqueDictionary['critique_Cat_list'] = templist

                #Update critiqued restaurant list according to cuisine type 
                templist = critiqueDictionary['critiqued_Res_AccrdCuisine_list']
                templist.remove(matchedResList)
                critiqueDictionary['critiqued_Res_AccrdCuisine_list'] = templist

                #UPDATE CRITIQUE RESTAURANT LIST 
                #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict']\
                                                           if item not in matchedResList or \
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list']+ \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]
            
            tempRecommendList = [item for item in matchedResList if item not in critiqueDictionary['critique_Res_list']]
            
            #Check if exhausted recommendation list 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:    
                #Update current valid set, make sure not in critiqued restaurant set 
                critiqueDictionary['current_user_item_predict'] = tempRecommendList.copy()
            
        #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
        #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
        if 'price' in critique_feature:
            
            #Get current price label range in 1 - 4
            currentPriceLabel = IP_dictionary[critiqueDictionary['current_user_item_predict'][recommendIndex]]
            
            #Get critique_price_list first
            templist = critiqueDictionary.get('critique_Price_list', []).copy()
            
            #Update critiqued price list
            if 'positive' in critique_posNeg and critique_value == 'more expensive':
                templist.extend([price for price in range(1,currentPriceLabel+1,1)])
                
            #Negative or positive, cheaper     
            else:     
                templist.extend([price for price in range(currentPriceLabel,5,1)])
            
            #update critiqued_price_list by puting it back to the dictionary
            critiqueDictionary['critique_Price_list'] = list(set(templist))

            #Check if critiqued all price range, if exhausted, fall back 
            if len(critiqueDictionary['critique_Price_list']) == 4:
                print('You have exhaused the price range option, showing the most', critique_value, 'option')
                #Send feedback to front end
                additionalText = 'You have exhaused the price range option, showing the most '+ critique_value + ' option'
                
                #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
                #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdStar_list'] + \
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))]

                #Clear out the restuarants critiquied by price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = []

                #Restore default critiquing price list 
                if critique_value == 'more expensive':
                    critiqueDictionary['critique_Price_list'] = [1,2,3]
                else:
                    critiqueDictionary['critique_Price_list'] = [2,3,4]  
                    
            #printing and sending message to front end, the critiquing price range
            print('Critiqing price at range:', ['$'*label for label in critiqueDictionary['critique_Price_list'] ])
            #additionalText += 'Critiqing price at range:'+str(['$'*label for label in critiqueDictionary['critique_Price_list']])
            
            #Find the list of restaurants to critique 
            listCritiqueRestaurant = [key for (key, value) in IP_dictionary.items() if \
                                       value in critiqueDictionary['critique_Price_list']]
            
            #find temporary updated critique list    
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRestaurant)
            
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in tempCritiqueList]
            
            #CHECK IF EXHAUSTED, if so clear out all restaurants 
            if exhaustFallBack(tempRecommendList):
                additionalText = additionalText +'| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            
            else:
                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = tempCritiqueList.copy()

                #Record the list of restaurants critiqued so far based on price 
                critiqueDictionary['critiqued_Res_AccrdPrice_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdPrice_list'], \
                                                           listCritiqueRestaurant)
                
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')

                ##Update current valid set , filter out critiqued items, sequence must remain the same
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                                   if item not in tempCritiqueList]
            
        #Scenario 6 negatively critique rating & positively critique rating: both goes up 
        #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
        #I will critique the ratings below the current rating or specific rating 
        if 'rating' in critique_feature:
            currentRating = float(recommendResDict['recBusinessJson']['rating'])
            
            if critique_value.lower() == 'current_rating':
                critique_value = currentRating
        
            #Critiquing the restaurants that has ratings equal and below this restaurant 
            critiqueRes = [rating/10 for rating in range(0,int(float(critique_value)*10)+1,5)]
            critiqueDictionary['ciritiqued_Rating_list'] = updateExtendList(critiqueDictionary['ciritiqued_Rating_list'], \
                                                                           critiqueRes)
            #If critiqued all price
            if critiqueDictionary['ciritiqued_Rating_list'] == [price/10 for price in range(0,51,5)]:
                print('exhausted list, no better restaurants, recommending the finest restaurants')
                additionalText += '| Exhausted list, no better restaurants, recommending restaurants beyond current rating'

                #reset critiqued restaurant list 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    item in list(set(critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdCuisine_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                     critiqueDictionary['critiqued_Res_AccrdDistance_list']))] 

                #Clear out the restuarants critiquied by rating 
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = []
                critiqueDictionary['ciritiqued_Rating_list'] = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

            print('critiquing restaurants at rating rangeing at:',critiqueDictionary['ciritiqued_Rating_list'])
            #additionalText += '| critiquing restaurants at rating rangeing at:'+str(critiqueDictionary['ciritiqued_Rating_list'])

            #Get the list of restaurants to critique
            listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if \
                               value in critiqueDictionary['ciritiqued_Rating_list']]
            
            #CHECK EXHAUSTED - Check to see if user critique exhausts the recommendation list
            #Take a temporary list as updated critiquing list 
            tempCritiqueList = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)
            
            #Take a temporary list as updated recommendation list 
            tempRecommendList = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                               if item not in tempCritiqueList]
            #Exhausted recommendation list
            if exhaustFallBack(tempRecommendList):
                additionalText += '| Exhausted recommendation list, no matching restaurant after current critique, discarding current critique'
            else:
                #Updating the critiqued items to list
                print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

                #Update the critiqued restaurant list
                critiqueDictionary['critique_Res_list'] = updateExtendList(critiqueDictionary['critique_Res_list'], listCritiqueRes)

                #Record those as well
                critiqueDictionary['critiqued_Res_AccrdStar_list'] = updateExtendList(critiqueDictionary['critiqued_Res_AccrdStar_list'], \
                                                                                      listCritiqueRes)
                #Update current valid set 
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        
        #Scenario #7 Critique on Distance, goes up or down, only say positive critiques. Want closer, further, or specific value 
        if 'distance' in critique_feature:
            
            #Get current distance
            currentDistance = float(recommendResDict['recBusinessJson']['distance'].strip('km'))
            
            #If want further distance restaurants
            if 'positive' in critique_posNeg and 'further' in critique_value:
                print('Critiquing distance <=', currentDistance)
                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]

            #Closer distance, critique value not empty 
            else:
                if (critique_value) and not 'closer' in critique_value:
                    currentDistance = float(critique_value)

                print('Critiquing distance >=', currentDistance)

                #New requirements for restaurants to be critiqued
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]


            #Update critique restaurant list, only keep the ones that are critiqued under other features
            critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                                      (item in list(set(listCritiquedRestaurant+\
                                                                        critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                                        critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))] 

            #Update list of restaurants critiqued by the restuarnt distance, entire replacement
            critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

            #Update initial valid restuarnt list
            critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict']\
                                                               if item not in critiqueDictionary['critique_Res_list']]

            #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
            if(len(critiqueDictionary['current_user_item_predict']) == 0):
                print('Exhausted sytem')

                #new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')
                additionalText += 'No restaurants that matches your preference within this area, currently default back to 2.0km'
                new_Distance = 2.0
                
                listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]

                #Restore original critiqued restaurants under distance feature 
                critiqueDictionary['critique_Res_list'] = [item for item in critiqueDictionary['test_user_item_predict'] if\
                                    (item in list(set(listCritiquedRestaurant + \
                                                      critiqueDictionary['critiqued_Res_AccrdPrice_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdStar_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdName_list']+\
                                                      critiqueDictionary['critiqued_Res_AccrdCuisine_list'])))]   
                #New critiquing restuarants
                critiqueDictionary['critiqued_Res_AccrdDistance_list'] = listCritiquedRestaurant

                #Update initial valid restuarnt list
                critiqueDictionary['current_user_item_predict'] = [item for item in critiqueDictionary['current_user_item_predict'] \
                                                                   if item not in critiqueDictionary['critique_Res_list']]
        
        #Full fall back 
        if(len(critiqueDictionary['current_user_item_predict']) ==0):
            fallBackSetup(critique_Res_list=critiqueDictionary['critique_Res_list'], \
                              critiqued_Res_AccrdName_list=critiqueDictionary['critiqued_Res_AccrdName_list'], \
                              critiqueDistance=2.0)
            additionalText += ' Ran out of recommendations at the end of the process, reset, filtered out seen restaurants'
            
            #If still no results
            if len(critiqueDictionary['current_user_item_predict']) ==0:
                fallBackSetup(critique_Res_list=[], \
                              critiqued_Res_AccrdName_list=[], \
                              critiqueDistance=2.0)
                additionalText += ' Clearing EVERYTHING'

        #Construct newly returning item
        return_Json = constructRestaurantDict(recommendIndex, critiqueDictionary['current_user_item_predict'],\
                                              df,ID_dictionary,additionalText)

        #Update restaurant information for the GET method
        recommendResDict['recBusinessJson'] = return_Json.copy()

        #Return newly computed object 
        return {'You sent': request_json, 'Result': return_Json}, 201
            
api.add_resource(Business, '/business') # Route_1

if __name__ == '__main__':
     app.run(port='5002')

{'business_id': '2970', 'name': 'Khao San Road', 'cuisine': 'Thai', 'price': '$$', 'rating': '4.0', 'distance': '0.8km', 'addText': ''}
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5003/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Feb/2020 22:56:34] "GET /business HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2020 22:56:34] "GET /favicon.ico HTTP/1.1" 404 -


## Original critiquing logic
need to update updateExtendList function 
need to check when calling exhaust list, if retrieving message 


In [68]:
print('Initial Recommendation')
print('Enter Stop any time to exist loop')
#displayRestaurantInfo(recommendIndex, test_user_item_predict)

while True:
    satisfied = 'None'
    feature = 'None'
    positiveOrNegative = 'None'
    critiqueValue = 'None'
    
    print('Recommending...')
    displayRestaurantInfo(recommendIndex, current_user_item_predict,df,ID_dictionary)
    
    #Current recommendation cuisine type 
    currentCuisineType = df[df['business_num_id'] == current_user_item_predict[recommendIndex]]\
                                                        .categories.unique()[0].split(', ')
    currentPriceLabel = IP_dictionary[current_user_item_predict[recommendIndex]]
    currentRating = IS_dictionary[current_user_item_predict[recommendIndex]]
    currentDistance = ID_dictionary[current_user_item_predict[recommendIndex]]
    
    #First testing cuisine type
    print('\n????????????????')
    
    while satisfied.lower() not in ['yes', 'no', 'stop']: 
        satisfied = input("You Like? ('yes', 'no') ").strip().lower()
    
    if satisfied == 'stop' or satisfied == 'yes':
        print('BYE :)')
        break
        
    #When satisfied is NO, take in feature
    while feature.lower() not in ['name', 'cuisine', 'price', 'distance', 'rating', 'stop']: 
        feature = input("What feature to critique: (name, cuisine, price, rating, distance)")
    
    if feature == 'stop':
        break
    
    #Take in Positive or nagative 
    if feature != 'name':
        while positiveOrNegative.lower() not in ['positive', 'negative', 'stop']:
            positiveOrNegative = input("Positive or negative: ")
    
    if positiveOrNegative == 'stop':
        break
    
    #Only ask for critique value when not critiquing restaurant name, or not negatively critiquing price 
    if feature != 'name' and not(feature == 'price' and positiveOrNegative == 'negative')\
    and not(feature == 'distance' and positiveOrNegative == 'negative'):
    #and not(feature == 'rating' and positiveOrNegative == 'negative')\
     
        #The valid values to be critiuqed that can pass in 
        validCritiqueValueList = []
        #negatively critique current cuisine type, should only enter current cruisine type
        if 'cuisine' in feature and 'negative' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in currentCuisineType] + ['stop']
            outPutString = '(' + currentCuisineType[0] +')'
        elif 'cuisine' in feature and 'positive' in positiveOrNegative:
            validCritiqueValueList = [cat.strip().lower() for cat in categoryList] 
            categories = list(set(IC_dictionary.keys())) + ['stop']
            outPutString = '(' + categories[0] +categories[1] +categories[2] +'...)'
        #Can only enter cheapter or more expensive
        elif 'price' in feature:
            validCritiqueValueList = ['cheaper', 'more expensive','stop']
            outPutString = '(cheaper, more expensive)'
        elif 'rating' in feature:
            validCritiqueValueList = [str(star/10) for star in range(0,51,1)] + ['stop', 'current_rating']
            outPutString = '(0 ~ 5.0 with 0.1 increment)'
        #positively critique distance
        elif 'distance' in feature:
            validCritiqueDis = [str(i/10) for i in range(0,int((variableDictionary['maxDistance']+0.5)*10),5)][1:] 
            validCritiqueValueList = ['closer', 'further'] + validCritiqueDis + ['stop']
            outPutString = '(closer, further, or distance in range 0.5 ~' + str(variableDictionary['maxDistance']) + 'in every 0.5km)' 
        #Prompt to ask critique value
        while critiqueValue not in validCritiqueValueList:
            critiqueValue = input("Critique value: " + outPutString).strip().lower()
        if critiqueValue == 'stop':
            break
            
    print('????????????????\n')
    
    #If user starts critiquing, the current showend restaurant will be in critiqued list 
    critique_Res_list.append(current_user_item_predict[recommendIndex])
    critique_Res_list = list(set(critique_Res_list))
    #critique_Res_list = updateList(critique_Res_list, current_user_item_predict[recommendIndex])
    
    #Scenario 1 - Critique Restaurant name 
    if 'name' in feature:
        
        print("Saving critiqued item: ", current_user_item_predict[recommendIndex])
        
        #Save critiqued restaurant to list 
        #critique_Res_list.append(current_user_item_predict[recommendIndex])
        #critique_Res_list = list(set(critique_Res_list))
        #critique_Res_list = updateList(critique_Res_list, current_user_item_predict[recommendIndex])
        
        #update critiqued restaurant list according to name
        #critiqued_Res_AccrdName_list.append(current_user_item_predict[recommendIndex])
        critiqued_Res_AccrdName_list = updateList(critiqued_Res_AccrdName_list, current_user_item_predict[recommendIndex])
        
        current_user_item_predict = current_user_item_predict[recommendIndex+1 :]
        #Handling all items critiqued case
        """BUT I'M NOT COUNTING THE FACTOR THAT THE USER HAD LIKED CATEGORIES"""
        current_user_item_predict, message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
            
    #Scenario 2 - Negatively critique restaurant features 
    if ('cuisine' in feature.lower()) and ('negative' in positiveOrNegative.lower()):
        
        #Find the correct category name the user want to critique in current recommended item categories
        critiqueValue = [cuisine for cuisine in currentCuisineType if critiqueValue.strip().lower() in cuisine.lower()][0]
        
        critiquied_Cat_Index = IC_dictionary[critiqueValue]
        print('Saving negatively critiqued cuisine type:', critiqueValue, ', cuisine index: ', critiquied_Cat_Index)
        critique_Cat_list.append(critiquied_Cat_Index)
        
        #Handling extreme case - critique categories previously requested 
        if critiquied_Cat_Index == wanted_Category_index:
            #Reset wanted category index
            wanted_Category_index = None
            print('You are critiquing a cuisine type you previously requested\n Fall back to initial preference')
            current_user_item_predict = [item for item in test_user_item_predict if item not in critique_Res_list]
        
        
        #Find list of restaurants to filter out 
        critiquedItemsList = I_C_matrix.getcol(critiquied_Cat_Index).nonzero()[0]
        
        critiqued_Res_AccrdCuisine_list.append(critiquedItemsList)
        critiqued_Res_AccrdCuisine_list =list(set(critique_Res_list))
        
        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', critiquedItemsList[:5], '...')
        critique_Res_list.extend(list(critiquedItemsList))
        critique_Res_list = list(set(critique_Res_list))
        
        #Filter out critiqued items, sequence must remain the same 
        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]

        #Handle case where run out of items! - Fall Back!
        current_user_item_predict,message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
        
        
    
    #Scenario 3 - Positively critique restaurant cuisine type 
    if 'cuisine' in feature and 'positive' in positiveOrNegative:

        #Find the correct category within all the categories list - assuming exact word typed in 
        """Add ERROR HANDLING HERE"""
        positiveCritiquiCategory = [cuisine for cuisine in categoryList if critiqueValue.strip().lower() in cuisine.lower()][0]
        
        #Retrieve cuisine type index 
        positiveCritiquiedIndex = IC_dictionary[positiveCritiquiCategory]
        
        #Get the preferred category index 
        wanted_Category_index = positiveCritiquiedIndex
        
        #Retrieve items matching item category 
        matchedResList = I_C_matrix.getcol(positiveCritiquiedIndex).nonzero()[0]
        print('matching list:', matchedResList[:10])
        
        #Check if user have previously critiqued - remove it
        if positiveCritiquiedIndex in critique_Cat_list:
            critique_Cat_list.remove(positiveCritiquiedIndex)
            
            #Update critiqued restaurant list according to cuisine type 
            critiqued_Res_AccrdCuisine_list.remove(matchedResList)
            
            #UPDATE CRITIQUE RESTAURANT LIST 
            #In the matched restuarnt list and not critiqued under other criteria will be withdrawn from this list
            critique_Res_list = [item for item in critique_Res_list if item not in matchedResList or \
                                item in list(set(critiqued_Res_AccrdName_list+ critiqued_Res_AccrdCuisine_list+\
                                                 critiqued_Res_AccrdStar_list + critiqued_Res_AccrdDistance_list))]
        
        #Update current valid set, make sure not in critiqued restaurant set 
        current_user_item_predict = [item for item in matchedResList if item not in critique_Res_list]
        
    #Scenario 4 - Negative critique restuarant price e.g. "I don't want expensive restaurants" OR positive "I want cheaper"
    #Does not pass in anything ASSUMING ONLY GOING DOWN
    #Scenario 5 - Positively critique restaurant price, "I want fine dining", wanting more expensive restaurant 
    if 'price' in feature:
        
        #Update critiqued price list
        if 'positive' in positiveOrNegative and critiqueValue == 'more expensive':
            critique_Price_list.extend([price for price in range(1,currentPriceLabel+1,1)])
        #Negative or positive, cheaper     
        else:     
            critique_Price_list.extend([price for price in range(currentPriceLabel,5,1)])
            
        #Deduplicate
        critique_Price_list = list(set(critique_Price_list))
        
        #Check if critiqued all price range, if exhausted, fall back 
        if len(critique_Price_list) == 4:
            print('You have exhaused the price range option, showing the most', critiqueValue, 'option')
            #Clear the previous critiqued restuarants based on price out of critiqued restuarant list 
            #POTENTIAL ISSUE HERE, MAY BE ERASING SOME RESTAURANTS CRITIQUED IN OTHER STEPS ... NEED TO RECHECK
            critique_Res_list = [item for item in test_user_item_predict if\
                                item in list(set(critiqued_Res_AccrdName_list+ critiqued_Res_AccrdCuisine_list+\
                                                 critiqued_Res_AccrdStar_list + critiqued_Res_AccrdDistance_list))]
            
            #Clear out the restuarants critiquied by price 
            critiqued_Res_AccrdPrice_list = []
            
            #Restore default critiquing price list 
            if critiqueValue == 'more expensive':
                critique_Price_list = [1,2,3]
            else:
                critique_Price_list = [2,3,4]  
        
        print('Critiqing price at range:', ['$'*label for label in critique_Price_list])
        
        #Find the list of restaurants to critique 
        listCritiqueRestaurant = [key  for (key, value) in IP_dictionary.items() if value in critique_Price_list]
        
        #Record the list of restaurants critiqued so far based on price 
        #critiqued_Res_AccrdPrice_list.extend(listCritiqueRestaurant)
        #critiqued_Res_AccrdPrice_list = list(set(critiqued_Res_AccrdPrice_list))
        
        critiqued_Res_AccrdPrice_list = updateExtendList(critiqued_Res_AccrdPrice_list, listCritiqueRestaurant)
        
        #Update the critiqued restaurant list
        #critique_Res_list.extend(listCritiqueRestaurant)
        #critique_Res_list = list(set(critique_Res_list))
        critique_Res_list = updateExtendList(critique_Res_list, listCritiqueRestaurant)
        
        
        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', listCritiqueRestaurant[:5], '...')
        
        #Filter out critiqued items, sequence must remain the same 
        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]
        
        #Check if exhaused list
        current_user_item_predict, message = checkIfExhaustedList(current_user_item_predict, test_user_item_predict, critique_Res_list)
        
        #Recommended
        print('Re-recommending...')
        
    #Scenario 6 negatively critique rating & positively critique rating: both goes up 
    #"I don't want ratings this low?" "I don't want restuarants with rating below XXX" "I want restaurants with rating above XXX"
    #I will critique the ratings below the current rating or specific rating 
    if 'rating' in feature:
        #Critiquing the restaurants that has ratings equal and below this restaurant 
        #if 'negative' in positiveOrNegative:
            #ciritiqued_Rating_list.extend([rating/10 for rating in range(0,int(currentRating*10+1),5)])
        #else:
        
        if critique_value.lower() == 'current_rating':
                critique_value = currentRating

        ciritiqued_Rating_list.extend([rating/10 for rating in range(0,int(float(critiqueValue)*10)+1,5)])   
        ciritiqued_Rating_list = list(set(ciritiqued_Rating_list))
            
        #If critiqued all price
        if ciritiqued_Rating_list == [price/10 for price in range(0,int(5.0 *10 +1),5)]:
            print('exhausted list, no better restaurants, recommending he finest restaurants')
            #reset critiqued restaurant list 
            critique_Res_list = [item for item in test_user_item_predict if\
                                item in list(set(critiqued_Res_AccrdName_list+critiqued_Res_AccrdCuisine_list+\
                                                critiqued_Res_AccrdPrice_list+critiqued_Res_AccrdDistance_list))] 

            #Clear out the restuarants critiquied by rating 
            critiqued_Res_AccrdStar_list = []

            ciritiqued_Rating_list = [rating/10 for rating in range(0,int(currentRating*10+1),5)]

        print('critiquing restaurants at rating rangeing at:',ciritiqued_Rating_list)

        #Get the list of restaurants to critique
        listCritiqueRes = [key  for (key, value) in IS_dictionary.items() if value in ciritiqued_Rating_list]

        #Updating the critiqued items to list
        print('Saving critiqued items at index: ', listCritiqueRes[:5], '...')

        #Update the critiqued restaurant list
        #critique_Res_list.extend(listCritiqueRes)
        #critique_Res_list = list(set(critique_Res_list))
        critique_Res_list = updateExtendList(critique_Res_list, listCritiqueRes)
        
        #Record those as well
        #critiqued_Res_AccrdStar_list.extend(listCritiqueRes)
        #critiqued_Res_AccrdStar_list = list(set(critiqued_Res_AccrdStar_list))
        critiqued_Res_AccrdStar_list = updateExtendList(critiqued_Res_AccrdStar_list, listCritiqueRes)

        #Update current valid set 
        current_user_item_predict = [item for item in current_user_item_predict if item not in critique_Res_list]
        
        
    #Scenario #7 Critique on Distance, goes up or down, only say positive critiques
    #Want closer, further, or specific value 
    if 'distance' in feature:
        #If want further distance restaurants
        if 'positive' in positiveOrNegative and 'further' in critiqueValue:
            print('Critiquing distance <=', currentDistance)

            #New requirements for restaurants to be critiqued
            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value <= currentDistance]
        
        #Closer distance
        else:
            if (critiqueValue != 'None') and not 'closer' in critiqueValue:
                currentDistance = int(critiqueValue)
                
            print('Critiquing distance >=', currentDistance)
            
            #New requirements for restaurants to be critiqued
            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value >= currentDistance]
        

        #Update critique restaurant list, only keep the ones that are critiqued under other features
        critique_Res_list = [item for item in test_user_item_predict if\
                            (item in list(set(listCritiquedRestaurant+critiqued_Res_AccrdPrice_list+\
                                              critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list+\
                                                critiqued_Res_AccrdCuisine_list)))] 
        
        #Update list of restaurants critiqued by the restuarnt distance, entire replacement
        critiqued_Res_AccrdDistance_list= listCritiquedRestaurant
        
        #Update initial valid restuarnt list
        current_user_item_predict = [item for item in  current_user_item_predict if item not in critique_Res_list]
        
        #Handle case where critiqued everything, there's no restuarnt that satisfy all the critiques 
        if(len(current_user_item_predict) == 0):
            print('Exhausted sytem')
            
            new_Distance= input('There\'s no restaurant that matches your preference within this area, please input a new larger distance')

            listCritiquedRestaurant = [key for (key, value) in ID_dictionary.items() if value > new_Distance]
            
            #Restore original critiqued restaurants under distance feature 
            critique_Res_list = [item for item in test_user_item_predict if\
                                (item in list(set(listCritiquedRestaurant+critiqued_Res_AccrdPrice_list\
                                                  +critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list+\
                                                    critiqued_Res_AccrdCuisine_list)))] 
            
            #New critiquing restuarants
            critiqued_Res_AccrdDistance_list= listCritiquedRestaurant
        
            #Update initial valid restuarnt list
            current_user_item_predict = [item for item in  current_user_item_predict if item not in critique_Res_list]
        
    

Initial Recommendation
Enter Stop any time to exist loop
Recommending...


NameError: name 'recommendIndex' is not defined

In [ ]:
current_user_item_predict

In [ ]:
merge = listCritiquedRestaurant+critiqued_Res_AccrdPrice_list+ critiqued_Res_AccrdStar_list+critiqued_Res_AccrdName_list + critiqued_Res_AccrdCuisine_list
a = 4139
4139 not in critiqued_Res_AccrdDistance_list or 4139 in list(set(merge))

### Requests  Example

In [ ]:
from flask import Flask
from flask_restful import reqparse, abort, Api, Resource

app = Flask(__name__)
api = Api(app)

TODOS = {
    'todo1': {'task': 'build an API'},
    'todo2': {'task': '?????'},
    'todo3': {'task': 'profit!'},
}


def abort_if_todo_doesnt_exist(todo_id):
    if todo_id not in TODOS:
        abort(404, message="Todo {} doesn't exist".format(todo_id))

parser = reqparse.RequestParser()
parser.add_argument('task')


# Todo
# shows a single todo item and lets you delete a todo item
class Todo(Resource):
    def get(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        return TODOS[todo_id]

    def delete(self, todo_id):
        abort_if_todo_doesnt_exist(todo_id)
        del TODOS[todo_id]
        return '', 204

    def put(self, todo_id):
        args = parser.parse_args()
        task = {'task': args['task']}
        TODOS[todo_id] = task
        return task, 201


# TodoList
# shows a list of all todos, and lets you POST to add new tasks
class TodoList(Resource):
    def get(self):
        return TODOS

    def post(self):
        args = parser.parse_args()
        todo_id = int(max(TODOS.keys()).lstrip('todo')) + 1
        todo_id = 'todo%i' % todo_id
        TODOS[todo_id] = {'task': args['task']}
        return TODOS[todo_id], 201


## Actually setup the Api resource routing here

api.add_resource(TodoList, '/todos')
api.add_resource(Todo, '/todos/<todo_id>')


if __name__ == '__main__':
    app.run(debug=False)